<h1>Segmenting and Clustering Neighborhoods in Toronto </h1>


##### In case the maps do not show, please view the notebook on nbviewer
##### Link: https://nbviewer.jupyter.org/github/Walid-Bejjar/Coursera_Capstone/blob/main/.ipynb_checkpoints/Segmenting%20and%20Clustering%20Neighborhoods%20in%20Toronto-checkpoint.ipynb

## Importing the Libraries

In [22]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


## Part 1 : Creating the Dataframe

#### Saving the data from the wikipedia page in a dataframe

In [23]:
# Saving the table in the wikipedia page as a dataframe
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
df_website = pd.read_html(url)
df = df_website[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Removing rows where Borough is not assigned

In [24]:
df=df.replace('Not assigned',np.NaN)
df = df.dropna(subset=["Borough"], axis=0)
df.reset_index(drop=True, inplace=True)
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [25]:
df.loc[df["Neighbourhood"] == np.NaN] # To verify if a cell has a borough but a Not assigned neighborhood

,Postal Code,Borough,Neighbourhood


In [26]:
df.shape

(103, 3)

## Part 2: Adding the Latitude and Longitude

#### Reading the csv file into a dataframe "df_coordinates"

In [27]:
df_coordinates = pd.read_csv("http://cocl.us/Geospatial_data")
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Verifying the shape "df_coordinates". It should be similar to "df" (103,3) 

In [28]:
df_coordinates.shape

(103, 3)

#### Sorting both dataframes by "Postal Code" and adding the Latitude and Longitude to "df" to have the complete dataframe

In [29]:
df=df.sort_values(by=['Postal Code'])
df.reset_index(drop=True, inplace=True)
df_coordinates = df_coordinates.sort_values(by=['Postal Code'])
df_coordinates.reset_index(drop=True, inplace=True)
df["Latitude"] = df_coordinates["Latitude"]
df["Longitude"] = df_coordinates["Longitude"]
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [30]:
df.shape

(103, 5)

## Part 3

### The objective of this project is to explore Toronto and divide its neighbourhoods into clusters. Since some postal codes areas contain several neighbouhoods, I will be using Postal Code to make it easier to visualize.
### 3 Clusters will be created based on the 4 most common venues in each postal code area. The common venues will be obtained via Foursquare.  

#### Creating a map of toronto with all postal code areas superimposed 

In [31]:
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [32]:
# Creating a map of Toronto using Folium
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
# Adding Markers
for lat, lng, borough, postal_code in zip(df['Latitude'], df['Longitude'], df['Borough'], df["Postal Code"]):
    label = '{}, {}'.format(postal_code, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Since M1X looks like the most remote area, I will be using it to determine the optimal radius of the search. When using a radius of less than 1000 meters, no results are found. I decided to use 2000 as a radius to get 4 venues.

In [33]:
CLIENT_ID = '2JBDXEOM4RNNJ3J4YFOAF5ROQHZO02QRQACZJCM3I53SBTNS' 
CLIENT_SECRET = 'LH1KEB0CXTIOUKBX0DECR5KCHVKJA3NLZCH5KXITOD5WD2EH' 
VERSION = '20180605' 
LIMIT = 100 
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2JBDXEOM4RNNJ3J4YFOAF5ROQHZO02QRQACZJCM3I53SBTNS
CLIENT_SECRET:LH1KEB0CXTIOUKBX0DECR5KCHVKJA3NLZCH5KXITOD5WD2EH


In [34]:
df.loc[df['Postal Code'] == 'M1X' ]

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
16,M1X,Scarborough,Upper Rouge,43.836125,-79.205636


In [35]:
postal_code_latitude = df.loc[16, 'Latitude'] 
postal_code_longitude = df.loc[16, 'Longitude'] 
postal_code_name = df.loc[16, 'Postal Code'] 
print('Latitude and longitude values of {} are {}, {}.'.format(postal_code_name, 
                                                               postal_code_latitude, 
                                                               postal_code_longitude))

Latitude and longitude values of M1X are 43.836124700000006, -79.20563609999999.


#### Obtaining the most common venues in the postal code area M1X

In [36]:
LIMIT = 100 
radius = 2000 
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    postal_code_latitude, 
    postal_code_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=2JBDXEOM4RNNJ3J4YFOAF5ROQHZO02QRQACZJCM3I53SBTNS&client_secret=LH1KEB0CXTIOUKBX0DECR5KCHVKJA3NLZCH5KXITOD5WD2EH&v=20180605&ll=43.836124700000006,-79.20563609999999&radius=2000&limit=100'

In [37]:
results = requests.get(url).json()

In [38]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [39]:
venues = results['response']['groups'][0]['items']
nearby_venues = pd.json_normalize(venues)
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Pumpkinland (Whittamore Farms),Farm,43.849236,-79.206647
1,Rouge Park - Finch Meander Trail,Trail,43.824850,-79.193596
2,Clownfish Village,Sculpture Garden,43.821000,-79.205396
3,Cedar Brae Golf and Country Club,Golf Course,43.839154,-79.230089


In [40]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


#### Obtaining the most common venues for all postal code areas

In [41]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code', 
                  'Postal Code Latitude', 
                  'Postal Code Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [42]:
toronto_venues = getNearbyVenues(names=df['Postal Code'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

M1B
M1C
M1E
M1G
M1H
M1J
M1K
M1L
M1M
M1N
M1P
M1R
M1S
M1T
M1V
M1W
M1X
M2H
M2J
M2K
M2L
M2M
M2N
M2P
M2R
M3A
M3B
M3C
M3H
M3J
M3K
M3L
M3M
M3N
M4A
M4B
M4C
M4E
M4G
M4H
M4J
M4K
M4L
M4M
M4N
M4P
M4R
M4S
M4T
M4V
M4W
M4X
M4Y
M5A
M5B
M5C
M5E
M5G
M5H
M5J
M5K
M5L
M5M
M5N
M5P
M5R
M5S
M5T
M5V
M5W
M5X
M6A
M6B
M6C
M6E
M6G
M6H
M6J
M6K
M6L
M6M
M6N
M6P
M6R
M6S
M7A
M7R
M7Y
M8V
M8W
M8X
M8Y
M8Z
M9A
M9B
M9C
M9L
M9M
M9N
M9P
M9R
M9V
M9W


#### Checking the number of venues for each postal code area

In [43]:
toronto_venues.groupby('Postal Code').count()

,Postal Code Latitude,Postal Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postal Code,,,,,,
M1B,48,48,48,48,48,48
M1C,38,38,38,38,38,38
M1E,41,41,41,41,41,41
M1G,63,63,63,63,63,63
M1H,100,100,100,100,100,100
M1J,73,73,73,73,73,73
M1K,53,53,53,53,53,53
M1L,88,88,88,88,88,88
M1M,37,37,37,37,37,37


#### Encoding and getting the 4 most common venues for each postal code area

In [44]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add postal code column back to dataframe
toronto_onehot['Postal Code'] = toronto_venues['Postal Code'] 

# move postal code column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Postal Code,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Beer Bar,Beer Store,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,College Stadium,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Leather Goods Store,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Restaurant,Road,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,So

In [45]:
toronto_grouped = toronto_onehot.groupby('Postal Code').mean().reset_index()
toronto_grouped.shape

(103, 333)

In [46]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [47]:
num_top_venues = 4

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postal Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
postal_code_venues_sorted = pd.DataFrame(columns=columns)
postal_code_venues_sorted['Postal Code'] = toronto_grouped['Postal Code']

for ind in np.arange(toronto_grouped.shape[0]):
    postal_code_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

postal_code_venues_sorted.shape

(103, 5)

#### Creating the clusters and visualizing them in the map

In [48]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Postal Code', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])

In [49]:
# add clustering labels
postal_code_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [50]:
# merge toronto_grouped with df to add latitude/longitude for each postal code area
toronto_merged = df
toronto_merged = toronto_merged.join(postal_code_venues_sorted.set_index('Postal Code'), on='Postal Code')
toronto_merged.head() 

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,0,Zoo Exhibit,Fast Food Restaurant,Park,Zoo
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0,Coffee Shop,Breakfast Spot,Bank,Pharmacy
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,Pizza Place,Breakfast Spot,Bank,Athletics & Sports
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Fast Food Restaurant,Sandwich Place,Supermarket
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Coffee Shop,Clothing Store,Gas Station,Restaurant


In [51]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postal Code'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [52]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[0] + list(range(5, toronto_merged.shape[1]))]]

,Postal Code,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
0,M1B,0,Zoo Exhibit,Fast Food Restaurant,Park,Zoo
1,M1C,0,Coffee Shop,Breakfast Spot,Bank,Pharmacy
2,M1E,0,Pizza Place,Breakfast Spot,Bank,Athletics & Sports
3,M1G,0,Coffee Shop,Fast Food Restaurant,Sandwich Place,Supermarket
4,M1H,0,Coffee Shop,Clothing Store,Gas Station,Restaurant
5,M1J,0,Coffee Shop,Fast Food Restaurant,Pizza Place,Sandwich Place
6,M1K,0,Pharmacy,Grocery Store,Chinese Restaurant,Coffee Shop
7,M1L,0,Fast Food Restaurant,Coffee Shop,Sandwich Place,Pizza Place
8,M1M,0,Harbor / Marina,Park,Coffee Shop,Fast Food Restaurant
9,M1N,0,Coffee Shop,Grocery Store,Pizza Place,Park


In [53]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[0] + list(range(5, toronto_merged.shape[1]))]]

,Postal Code,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
16,M1X,1,Trail,Sculpture Garden,Golf Course,Farm
36,M4C,1,Park,Café,Coffee Shop,Pizza Place
37,M4E,1,Coffee Shop,Pub,Beach,Breakfast Spot
38,M4G,1,Coffee Shop,Indian Restaurant,Bakery,Park
40,M4J,1,Greek Restaurant,Café,Gastropub,Coffee Shop
41,M4K,1,Café,Greek Restaurant,Bakery,Park
42,M4L,1,Park,Coffee Shop,Beach,Brewery
43,M4M,1,Coffee Shop,Park,Bakery,Café
44,M4N,1,Coffee Shop,Park,Sushi Restaurant,Italian Restaurant
45,M4P,1,Coffee Shop,Italian Restaurant,Park,Bakery


In [54]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[0] + list(range(5, toronto_merged.shape[1]))]]

,Postal Code,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
21,M2M,2,Korean Restaurant,Coffee Shop,Bubble Tea Shop,Café
22,M2N,2,Korean Restaurant,Grocery Store,Café,Supermarket
51,M4X,2,Coffee Shop,Café,Park,Diner
52,M4Y,2,Coffee Shop,Park,Café,Gastropub
53,M5A,2,Coffee Shop,Park,Restaurant,Café
54,M5B,2,Coffee Shop,Gastropub,Café,Theater
55,M5C,2,Coffee Shop,Café,Hotel,Gastropub
56,M5E,2,Coffee Shop,Café,Hotel,Plaza
57,M5G,2,Coffee Shop,Café,Park,Restaurant
58,M5H,2,Coffee Shop,Café,Park,Hotel


### Cluster 1 contains neighbourhoods that offer a big number of shops, stores and banks. Cluster 2 contains neighbourhooods that offer a variety of international cuisine restaurants and parks. Cluster 3 contains neighbourhoods that offer a big number of coffee shops and cafés.